In [1]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

# Embededstruct(int)

its used to cast embededstruct0 as currying funcion(typical in funcional programming EmbededStruct(int)(Struct,String="new"))

# EmbededStruct0(self::STruct,suffix::string="new")

it creates a copy of the self arg and it change the dimension with a factor of "n" and also append the suffix "new" on the name.and at the end return this copy

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
x = Lar.Struct([square])

LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "11465203679279825518", 2, "feature")

In [4]:
function embedStruct(n::Int)
	function embedStruct0(self::Lar.Struct,suffix::String="New")
		if n==0
			return self, length(self.box[1])
		end
		cloned = Lar.Struct()
		cloned.box = [ [self.body[i].box[1];zeros(dimadd)],
						[self.body[i].box[2];zeros(dimadd)] ]
		cloned.name = self.name*suffix
		cloned.category = self.category
		cloned.dim = self.dim+n
		cloned = Lar.embedTraversal(cloned,self,n,suffix)
		return cloned
	end
	return embedStruct0
end



embedStruct (generic function with 1 method)

In [5]:
@btime embedStruct(0)(x)
#error if non 0 cus it need to be debugged

  98.601 ns (1 allocation: 32 bytes)


(LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "13209065117907633244", 2, "feature"), 2)

In [7]:
@benchmark embedStruct(0)(x)

BenchmarkTools.Trial: 10000 samples with 952 evaluations.
 Range (min … max):   94.223 ns …  3.601 μs  ┊ GC (min … max): 0.00% … 96.32%
 Time  (median):      95.693 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   102.942 ns ± 83.718 ns  ┊ GC (mean ± σ):  1.95% ±  2.36%

  █▇▃▄▂▁ ▅▆▃▃▁▂▁▃▃▂     ▁▁                                     ▁
  ███████████████████▇█████▇▇▇▆▆▆▆▅▅▄▅▅▅▄▄▄▄▅▅▄▅▄▅▄▅▅▄▄▄▄▂▄▅▄▆ █
  94.2 ns       Histogram: log(frequency) by time       155 ns <

 Memory estimate: 32 bytes, allocs estimate: 1.

# Improvment and changes
worst performance but now work for n != 0

In [4]:
function embedStruct(n)
	function embedStruct0(self, suffix)
		if n==0
			return self, length(self.box[1])
		end
		cloned = Lar.Struct()
		cloned.box = hcat((self.box,[fill([0],n),fill([0],n)]))
		cloned.name = self.name*suffix
		cloned.category = self.category
		cloned.dim = self.dim+n
		cloned = Lar.embedTraversal(cloned,self,n,suffix)
		return cloned
	end
	return embedStruct0
end

embedStruct (generic function with 1 method)

In [5]:
@benchmark embedStruct(1)(x,"-New")

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  4.013 μs … 770.250 μs  ┊ GC (min … max): 0.00% … 97.93%
 Time  (median):     4.713 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.586 μs ±  14.962 μs  ┊ GC (mean ± σ):  5.24% ±  1.96%

  ▄▅▇█▇▅▄▄▄▃▃▃▂▁ ▁                                            ▂
  █████████████████▇▇▆▆▆▆▅▄▅▅▅▆▅▅▅▅▅▄▄▄▅▄▅▅▄▅▆▆▇▇▇▇▆▆▆▆▆▇▆▅▆▆ █
  4.01 μs      Histogram: log(frequency) by time      14.5 μs <

 Memory estimate: 2.45 KiB, allocs estimate: 33.

In [6]:
@benchmark embedStruct(0)(x,"-New")

BenchmarkTools.Trial: 10000 samples with 891 evaluations.
 Range (min … max):  112.907 ns …   4.799 μs  ┊ GC (min … max): 0.00% … 96.14%
 Time  (median):     133.109 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   144.960 ns ± 110.591 ns  ┊ GC (mean ± σ):  1.61% ±  2.34%

  ▆▆▇▇▇█▆▆▅▄▃▂▃▂▁▂▁▁▁                                           ▂
  █████████████████████████▇▇█▆▇▇▆▇▇▆▄▅▅▆▇▅▆▅▆▅▆▃▅▅▆▅▃▄▄▄▅▅▅▄▄▅ █
  113 ns        Histogram: log(frequency) by time        350 ns <

 Memory estimate: 32 bytes, allocs estimate: 1.